In [2]:
#source summer2winter_yosemite
from google.colab import drive
drive.mount('/content/drive')

!pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install -U tensorflow-addons
!pip install git+https://github.com/skrzypczykt/CycleGAN.git
!pip install matplotlib==3.1.1
%load_ext autoreload
%autoreload 2



# Download data 

!wget https://s3.amazonaws.com/video.udacity-data.com/topher/2018/November/5be66e78_summer2winter-yosemite/summer2winter-yosemite.zip
!unzip -q summer2winter-yosemite.zip
!rm summer2winter-yosemite.zip
!mv ./summer2winter_yosemite ./summer2winter-yosemite

Mounted at /content/drive
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-0or6ifu6
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-0or6ifu6
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp37-none-any.whl size=101065 sha256=6414e8b6c81ca401e8cf494e0fd14c4673e6123eedc25aeb79ce5962c10c5803
  Stored in directory: /tmp/pip-ephem-wheel-cache-onz5cnp3/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib
     |████████████████████████████████| 686kB 7.4MB/s 
  Cloning https://github.com/skrzypczykt/CycleGAN.git to /tmp/pip-req-build-82wev9s8
  Running command git clone -q https://github.com/skrzypczykt/CycleGAN.git /tmp/pip-req-build-82wev9s8
     |████████████████████████████████| 184kB 8.2MB/s 
     |████████████████████████████████| 10.3MB 47.3MB/s 
     |████████████████████████████████| 3.0MB 42.6MB/s 
     |████████████████████████████

     |████████████████████████████████| 13.1MB 220kB/s 
ERROR: measurements 0.1 has requirement matplotlib==3.4.2, but you'll have matplotlib 3.1.1 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: matplotlib 3.4.2
    Uninstalling matplotlib-3.4.2:
      Successfully uninstalled matplotlib-3.4.2


--2021-05-30 19:38:59--  https://s3.amazonaws.com/video.udacity-data.com/topher/2018/November/5be66e78_summer2winter-yosemite/summer2winter-yosemite.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.80.211
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.80.211|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 133783376 (128M) [application/zip]
Saving to: ‘summer2winter-yosemite.zip’

summer2winter-yosem 100%[===================>] 127.58M  38.9MB/s    in 3.3s    

2021-05-30 19:39:03 (38.9 MB/s) - ‘summer2winter-yosemite.zip’ saved [133783376/133783376]



In [3]:
import os
from gans.CycleGAN import load_data, CycleGAN, check_directories
import tensorflow as tf
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
import matplotlib.pyplot as plt


In [ ]:
data_dir = '/content/summer2winter-yosemite'
#data_dir = '../data/summer2winter_yosemite'

IMG_WIDTH = 256
IMG_HEIGHT = 256

RESTORE_TRAINING = False

#PROJECT_ROOT_DIR = "/content/drive/MyDrive/CycleGAN/resnet/"
PROJECT_ROOT_DIR = "../trainings/resnet/"

IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
checkpoint_path = os.path.join(PROJECT_ROOT_DIR, "checkpoints")

check_directories(PROJECT_ROOT_DIR)

#load data
train_winter, test_winter, train_summer, test_summer, sample_A, sample_B = load_data(data_dir, IMG_WIDTH, IMG_HEIGHT)


gan = CycleGAN(
    input_dim=(IMG_WIDTH, IMG_HEIGHT, 3)
    , learning_rate=0.0002
    , buffer_max_length=50
    , lambda_validation=1
    , lambda_reconstr=10
    , lambda_id=2
    , generator_type='resnet'
    , gen_n_filters=32
    , disc_n_filters=32
    , n_batches=961
)

mode = 'build'
# mode = 'load'

if mode == 'build':
    gan.save(checkpoint_path)
else:
    #TODO: check
    max_epoch = 32 #???????
    gan.load_weights(os.path.join(checkpoint_path, f'weights/weights-{max_epoch}.h5'))


EPOCHS = 1
PRINT_EVERY_N_BATCHES = 1000

"""##train"""

gan.train(tf.data.Dataset.zip((train_winter, train_summer))
          , run_folder=checkpoint_path
          , epochs=EPOCHS
          , test_A_file=sample_A
          , test_B_file=sample_B
          , batch_size=1
          , sample_interval=PRINT_EVERY_N_BATCHES)

"""##Loss"""

fig = plt.figure(figsize=(20, 10))

plt.plot([x[1] for x in gan.g_losses], color='green', linewidth=0.1)  # DISCRIM LOSS
# plt.plot([x[2] for x in gan.g_losses], color='orange', linewidth=0.1)
plt.plot([x[3] for x in gan.g_losses], color='blue', linewidth=0.1)  # CYCLE LOSS
# plt.plot([x[4] for x in gan.g_losses], color='orange', linewidth=0.25)
plt.plot([x[5] for x in gan.g_losses], color='red', linewidth=0.25)  # ID LOSS
# plt.plot([x[6] for x in gan.g_losses], color='orange', linewidth=0.25)

plt.plot([x[0] for x in gan.g_losses], color='black', linewidth=0.25)


plt.xlabel('batch', fontsize=18)
plt.ylabel('loss', fontsize=16)

plt.ylim(0, 5)

plt.show()


"""#Test"""

i = 0
for imgA, img_B in tf.data.Dataset.zip((test_winter, test_summer)):
    gan.sample_images(imgA, img_B, i, os.path.join(PROJECT_ROOT_DIR, "test_results"), None, None, training=False)
    i = i + 1

Found 962 files belonging to 1 classes.
Found 1231 files belonging to 1 classes.
Found 238 files belonging to 1 classes.
Found 309 files belonging to 1 classes.
[Epoch 0/1] [Batch 0/961] [D loss: 1.9733637869358063, acc: 43.701171875] [G loss: 18.873783111572266, adv: 3.568519949913025, recon:1.2750118970870972, id: ] time: 1.2775716185569763 


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


[Epoch 0/1] [Batch 1/961] [D loss: 2.439866155385971, acc: 54.443359375] [G loss: 16.033634185791016, adv: 2.951967477798462, recon:1.0903250575065613, id: ] time: 1.089208722114563 
[Epoch 0/1] [Batch 2/961] [D loss: 2.30619877576828, acc: 53.5888671875] [G loss: 16.74046516418457, adv: 2.7625592947006226, recon:1.1667582988739014, id: ] time: 1.1551611125469208 
[Epoch 0/1] [Batch 3/961] [D loss: 1.6906669735908508, acc: 48.095703125] [G loss: 15.161794662475586, adv: 2.1911078691482544, recon:1.080395519733429, id: ] time: 1.0833659172058105 
[Epoch 0/1] [Batch 4/961] [D loss: 0.9270612448453903, acc: 47.2412109375] [G loss: 15.616129875183105, adv: 1.7854543328285217, recon:1.1506231427192688, id: ] time: 1.1622223854064941 
[Epoch 0/1] [Batch 5/961] [D loss: 0.5959741249680519, acc: 49.5849609375] [G loss: 16.039264678955078, adv: 1.6810316443443298, recon:1.1984867751598358, id: ] time: 1.1866821646690369 
[Epoch 0/1] [Batch 6/961] [D loss: 0.6474842354655266, acc: 50.3173828125]